# Token Counter Notebook

This notebook allows you to input text, retrieve outputs, and calculate token counts using OpenAI, Gemini, Anthropic, and Grok APIs.

## 1. Install and Import Required Libraries

Install necessary libraries such as `openai`, `anthropic`, and others. Import required modules like `os` and `json`.

In [1]:
# Install required libraries (uncomment if running for the first time)
# !pip install openai anthropic google-generativeai requests tiktoken

import os
import json
import openai
import anthropic
from google import genai
import requests
import tiktoken

In [2]:
# Make output cells take full width
from IPython.core.display import HTML
HTML("<style>.jp-OutputArea-output { max-width: 100vw !important; }</style>")

## 2. Set Up API Keys

Set up environment variables or directly define API keys for OpenAI, Gemini, Anthropic, and Grok.

In [ ]:
# Set your API keys here or load them from environment variables

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")
ANTHROPIC_API_KEY = os.getenv("ANTHROPIC_API_KEY", "your-anthropic-api-key")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY", "your-gemini-api-key")
GROK_API_KEY = os.getenv("GROK_API_KEY", "your-grok-api-key")

# Set API keys for respective libraries if possible
openai.api_key = OPENAI_API_KEY

AttributeError: module 'google.genai' has no attribute 'configure'

## 3. Define Helper Functions for Token Counting

Write helper functions to calculate token counts for each API, ensuring compatibility with their respective tokenization methods.

In [ ]:
def count_openai_tokens(text, model="gpt-3.5-turbo"):
    enc = tiktoken.encoding_for_model(model)
    return len(enc.encode(text))

def count_gemini_tokens(text):
    # Gemini uses SentencePiece tokenizer, but for simplicity, use whitespace split as a proxy
    return len(text.split())

def count_anthropic_tokens(text):
    # Anthropic uses Claude tokenizer, which is similar to GPT-3's
    enc = tiktoken.get_encoding("cl100k_base")
    return len(enc.encode(text))

def count_grok_tokens(text):
    # Grok's tokenizer is not public; use whitespace split as a proxy
    return len(text.split())

## 4. Process Text with OpenAI API

Send the input text to the OpenAI API, retrieve the output, and calculate the token count using the helper function.

In [ ]:
def process_with_openai_response(prompt, model="gpt-4.1"):
    try:
        from openai import OpenAI
        client = OpenAI(api_key=OPENAI_API_KEY)
        response = client.responses.create(
            model=model,
            input=prompt
        )
        output = getattr(response, 'output_text', str(response))
        # Token usage may not be available; set to None
        return output, None, None
    except Exception as e:
        return f"OpenAI Responses API error: {str(e)}", None, None

## 5. Process Text with Gemini API

Send the input text to the Gemini API, retrieve the output, and calculate the token count using the helper function.

In [ ]:
def process_with_gemini(prompt, model="gemini-2.0-flash"):
    try:
        client = genai.Client(api_key=GEMINI_API_KEY)
        response = client.models.generate_content(
            model=model,
            contents=[prompt],  # contents must be a list
        )
        output = getattr(response, 'text', str(response))
        return output, None, None
    except Exception as e:
        return f"Gemini error: {str(e)}", None, None

# Example usage:
# output, input_tokens, output_tokens = process_with_gemini(prompt)
# print(f"Output: {output}\nInput tokens: {input_tokens}\nOutput tokens: {output_tokens}")

## 6. Process Text with Anthropic API

Send the input text to the Anthropic API, retrieve the output, and calculate the token count using the helper function.

In [ ]:
def process_with_anthropic(prompt, model="claude-3-7-sonnet-20250219"):
    try:
        import anthropic
        client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        message = client.messages.create(
            model=model,
            max_tokens=1024,
            messages=[{"role": "user", "content": prompt}]
        )
        output = message.content[0].text if hasattr(message.content[0], 'text') else str(message.content[0])
        input_tokens = getattr(message.usage, 'input_tokens', None)
        output_tokens = getattr(message.usage, 'output_tokens', None)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"Anthropic error: {str(e)}", None, None

# Example usage:
# output, input_tokens, output_tokens = process_with_anthropic(prompt)
# print(f"Output: {output}\nInput tokens: {input_tokens}\nOutput tokens: {output_tokens}")

## 7. Process Text with Grok API

Send the input text to the Grok API, retrieve the output, and calculate the token count using the helper function.

In [ ]:
def process_with_grok(prompt, model="grok-3-beta"):
    try:
        from openai import OpenAI
        client = OpenAI(
            api_key=GROK_API_KEY,
            base_url="https://api.x.ai/v1",
        )
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        output = completion.choices[0].message.content
        input_tokens = getattr(completion.usage, 'prompt_tokens', None)
        output_tokens = getattr(completion.usage, 'completion_tokens', None)
        return output, input_tokens, output_tokens
    except Exception as e:
        return f"Grok error: {str(e)}", None, None

# Example usage:
# output, input_tokens, output_tokens = process_with_grok(prompt)
# print(f"Output: {output}\nInput tokens: {input_tokens}\nOutput tokens: {output_tokens}")

## 8. Enter Your Prompt

Define your input text in the variable below. This prompt will be sent to all four APIs.

In [ ]:
# Enter your prompt here
prompt = 'Explain the theory of relativity in one paragraph.'

## 9. Run All APIs and Compare Results

This cell will send your prompt to OpenAI, Gemini, Anthropic, and Grok, then display their outputs and token counts.

In [ ]:
results = {}
# OpenAI (Responses API)
try:
    output, in_tok, out_tok = process_with_openai_response(prompt)
    results['OpenAI'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok}
except Exception as e:
    results['OpenAI'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None}
# Gemini
try:
    output, in_tok, out_tok = process_with_gemini(prompt)
    results['Gemini'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok}
except Exception as e:
    results['Gemini'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None}
# Anthropic
try:
    output, in_tok, out_tok = process_with_anthropic(prompt)
    results['Anthropic'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok}
except Exception as e:
    results['Anthropic'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None}
# Grok
try:
    output, in_tok, out_tok = process_with_grok(prompt)
    results['Grok'] = {'output': output, 'input_tokens': in_tok, 'output_tokens': out_tok}
except Exception as e:
    results['Grok'] = {'output': str(e), 'input_tokens': None, 'output_tokens': None}
import pandas as pd
df = pd.DataFrame.from_dict(results, orient='index')
display(df)

# Optionally, print each output separately
# for api, res in results.items():
#     print(f'--- {api} ---')
#     print(f'Output: {res["output"]}\nInput tokens: {res["input_tokens"]}\nOutput tokens: {res["output_tokens"]}\n')

In [ ]:
# Output all results and errors to a text file for easier reading
with open('api_results.txt', 'w', encoding='utf-8') as f:
    for api, res in results.items():
        f.write(f'--- {api} ---\n')
        f.write(f'Output: {res["output"]}\nInput tokens: {res["input_tokens"]}\nOutput tokens: {res["output_tokens"]}\n\n')
print('Results dumped to api_results_dump.txt')